In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
%cd '/content/gdrive/My Drive/Colab Notebooks/DL4H Project' 

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/DL4H Project


In [ ]:
df_adm = pd.read_csv('./Data/ADMISSIONS.csv')

In [ ]:
len(df_adm.groupby(['SUBJECT_ID']))

46520

In [ ]:
len(df_adm.groupby(['HADM_ID'])['HADM_ID'])

58976

In [ ]:
df_adm.ADMITTIME = pd.to_datetime(df_adm.ADMITTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
df_adm.DISCHTIME = pd.to_datetime(df_adm.DISCHTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
df_adm.DEATHTIME = pd.to_datetime(df_adm.DEATHTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')

In [ ]:
df_adm = df_adm.sort_values(['SUBJECT_ID','ADMITTIME'])
df_adm = df_adm.reset_index(drop = True)

In [ ]:
df_adm['NEXT_ADMITTIME'] = df_adm.groupby('SUBJECT_ID').ADMITTIME.shift(-1)

In [ ]:
df_adm['NEXT_ADMISSION_TYPE'] = df_adm.groupby('SUBJECT_ID').ADMISSION_TYPE.shift(-1)

In [ ]:
rows = df_adm.NEXT_ADMISSION_TYPE == 'ELECTIVE'
df_adm.loc[rows,'NEXT_ADMITTIME'] = pd.NaT
df_adm.loc[rows,'NEXT_ADMISSION_TYPE'] = np.NaN

In [ ]:
df_adm = df_adm.sort_values(['SUBJECT_ID','ADMITTIME'])

df_adm[['NEXT_ADMITTIME','NEXT_ADMISSION_TYPE']] = df_adm.groupby(['SUBJECT_ID'])[['NEXT_ADMITTIME','NEXT_ADMISSION_TYPE']].fillna(method = 'bfill')

In [ ]:
df_adm['DAYS_NEXT_ADMIT']=  (df_adm.NEXT_ADMITTIME - df_adm.DISCHTIME).dt.total_seconds()/(24*60*60)

In [ ]:
df_adm1 = df_adm[['SUBJECT_ID','HADM_ID','ADMITTIME','DISCHTIME','DAYS_NEXT_ADMIT','NEXT_ADMITTIME','ADMISSION_TYPE','DEATHTIME']]

In [ ]:
df_adm1['OUTPUT_LABEL'] = (df_adm1.DAYS_NEXT_ADMIT < 30).astype('int')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df_adm1['STAY_DAYS']=  (df_adm1.DISCHTIME - df_adm1.ADMITTIME).dt.total_seconds()/(24*60*60)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df_adm1.loc[df_adm1.DEATHTIME.notnull(), 'OUTPUT_LABEL'] = 0

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [ ]:
len(df_adm1[df_adm1.DEATHTIME.notnull()])

5854

In [ ]:
df_adm1.OUTPUT_LABEL.value_counts()

0    55790
1     3186
Name: OUTPUT_LABEL, dtype: int64

In [ ]:
# drop newborn 
df_adm1 = df_adm1[df_adm1['ADMISSION_TYPE']!='NEWBORN']

In [ ]:
df_adm1.OUTPUT_LABEL.value_counts()

0    48150
1     2963
Name: OUTPUT_LABEL, dtype: int64

In [ ]:
#drop death
df_adm1 = df_adm1[df_adm1.DEATHTIME.isnull()]

In [ ]:
df_adm1.OUTPUT_LABEL.value_counts()

0    42358
1     2963
Name: OUTPUT_LABEL, dtype: int64

In [ ]:
len(df_adm1.groupby('SUBJECT_ID')['SUBJECT_ID'].unique())

34560

# Split

In [ ]:
readmit_ID = df_adm1[df_adm1.OUTPUT_LABEL == 1].HADM_ID

In [ ]:
not_readmit_ID = df_adm1[df_adm1.OUTPUT_LABEL == 0].HADM_ID

In [ ]:
not_readmit_ID_use = not_readmit_ID.sample(n=len(readmit_ID), random_state=1)

In [ ]:
id_val_test_t=readmit_ID.sample(frac=0.2,random_state=1)
id_val_test_f=not_readmit_ID_use.sample(frac=0.2,random_state=1)

In [ ]:
id_train_t = readmit_ID.drop(id_val_test_t.index)

In [ ]:
id_train_f = not_readmit_ID_use.drop(id_val_test_f.index)

In [ ]:
id_val_t=id_val_test_t.sample(frac=0.5,random_state=1)
id_test_t=id_val_test_t.drop(id_val_t.index)

id_val_f=id_val_test_f.sample(frac=0.5,random_state=1)
id_test_f=id_val_test_f.drop(id_val_f.index)

In [ ]:
(pd.Index(id_test_t).intersection(pd.Index(id_train_t))).values

array([], dtype=int64)

In [ ]:
id_test = pd.concat([id_test_t, id_test_f])

In [ ]:
len(id_test)

594

In [ ]:
id_val = pd.concat([id_val_t, id_val_f])

In [ ]:
len(id_val)

592

In [ ]:
id_train = pd.concat([id_train_t, id_train_f])
train_id_label = pd.DataFrame(data = list(zip(id_train, [1]*len(id_train_t)+[0]*len(id_train_f))), columns = ['id','label'])

In [ ]:
id_val = pd.concat([id_val_t, id_val_f])
val_id_label = pd.DataFrame(data = list(zip(id_val, [1]*len(id_val_t)+[0]*len(id_val_f))), columns = ['id','label'])

In [ ]:
id_test = pd.concat([id_test_t, id_test_f])
test_id_label = pd.DataFrame(data = list(zip(id_test, [1]*len(id_test_t)+[0]*len(id_test_f))), columns = ['id','label'])

In [ ]:
(pd.Index(train_id_label.id).intersection(pd.Index(test_id_label.id))).values

array([], dtype=int64)

In [ ]:
len(train_id_label)

4740

In [ ]:
train_id_label.to_csv('./good_datasets/good_train_id_label.csv')

In [ ]:
test_id_label.to_csv('./good_datasets/good_test_id_label.csv')

In [ ]:
val_id_label.to_csv('./good_datasets/good_val_id_label.csv')

## Discharge Processing

In [ ]:
want=pd.read_csv('./readmission_task.csv')
#want=pd.read_csv('./data/discharge/train.csv')
#discharge summary task dataset 

In [ ]:
discharge_train = want[want.ID.isin(train_id_label.id)]

In [ ]:
len(discharge_train)

24396

In [ ]:
discharge_train.Label.value_counts()

1.0    13134
0.0    11262
Name: Label, dtype: int64

In [ ]:
discharge_val = want[want.ID.isin(val_id_label.id)]

In [ ]:
discharge_val.Label.value_counts()

1.0    1613
0.0    1424
Name: Label, dtype: int64

In [ ]:
discharge_test = want[want.ID.isin(test_id_label.id)]

In [ ]:
discharge_test.Label.value_counts()

1.0    1697
0.0    1366
Name: Label, dtype: int64

In [ ]:
len(test_id_label)

594

In [ ]:
df = pd.concat([not_readmit_ID_use, not_readmit_ID])
df = df.drop_duplicates(keep=False)

In [ ]:
(pd.Index(df).intersection(pd.Index(not_readmit_ID_use))).values

array([], dtype=int64)

In [ ]:
not_readmit_ID_more = df.sample(n=400, random_state=1)

In [ ]:
dis_train = want[want.ID.isin(not_readmit_ID_more)]

In [ ]:
(pd.Index(dis_train.ID).intersection(pd.Index(discharge_train.ID))).values

array([], dtype=float64)

In [ ]:
len(dis_train)

1849

In [ ]:
discharge_train_snippets = pd.concat([dis_train, discharge_train])

In [ ]:
discharge_train_snippets = discharge_train_snippets.sample(frac=1, random_state=1).reset_index(drop=True)

In [ ]:
discharge_train_snippets.Label.value_counts()

1.0    13134
0.0    13111
Name: Label, dtype: int64

In [ ]:
discharge_train_snippets.to_csv('./good_datasets/discharge/train.csv')

In [ ]:
discharge_val.to_csv('./good_datasets/discharge/val.csv')

In [ ]:
discharge_test.to_csv('./good_datasets/discharge/test.csv')

## Now For Early Notes

In [ ]:
## get unactionable admits for Test
## Training 3 days

In [ ]:
# want_early = pd.read_csv('./data/3days/less_3_days_notes.csv')
want_early = pd.read_csv('./less_3_days_notes.csv')
#early notes fold 1

In [ ]:
early_train = want_early[want_early.ID.isin(train_id_label.id)]

In [ ]:
early_train.Label.value_counts()

1.0    23901
0.0    19808
Name: Label, dtype: int64

In [ ]:
not_readmit_ID_more = df.sample(n=500, random_state=1)

In [ ]:
ear_train = want_early[want_early.ID.isin(not_readmit_ID_more)]

In [ ]:
ear_train.Label.value_counts()

0.0    4084
Name: Label, dtype: int64

In [ ]:
early_train_snippets = pd.concat([ear_train, early_train])

In [ ]:
(pd.Index(ear_train.ID).intersection(pd.Index(early_train.ID))).values

array([], dtype=float64)

In [ ]:
early_train_snippets = early_train_snippets.sample(frac=1, random_state=1).reset_index(drop=True)

In [ ]:
early_train_snippets.to_csv('./good_datasets/3days/train.csv')

In [ ]:
early_val = want_early[want_early.ID.isin(val_id_label.id)]

In [ ]:
early_val.Label.value_counts()

1.0    2986
0.0    2788
Name: Label, dtype: int64

In [ ]:
early_val.to_csv('./good_datasets/3days/val.csv')

In [ ]:
actionable_ID_3days = df_adm1[df_adm1['STAY_DAYS'] >= 3].HADM_ID

In [ ]:
test_actionable_id_label = test_id_label[test_id_label.id.isin(actionable_ID_3days)]

In [ ]:
early_test = want_early[want_early.ID.isin(test_actionable_id_label.id)]

In [ ]:
early_test.Label.value_counts()

1.0    2904
0.0    2537
Name: Label, dtype: int64

In [ ]:
early_test.to_csv('./good_datasets/3days/test.csv')

## Now for two days test set

In [ ]:
#want_2days = pd.read_csv('less_2_days_notes.csv')
want_2days = pd.read_csv('./less_2_days_notes.csv')

In [ ]:
actionable_ID_2days = df_adm1[df_adm1['STAY_DAYS'] >= 2].HADM_ID

In [ ]:
test_actionable_id_label_2days = test_id_label[test_id_label.id.isin(actionable_ID_2days)]

In [ ]:
early_test_2days = want_2days[want_2days.ID.isin(test_actionable_id_label_2days.id)]

In [ ]:
early_test_2days.Label.value_counts()

1.0    2197
0.0    1857
Name: Label, dtype: int64

In [ ]:
early_test_2days.to_csv('./good_datasets/2days/test.csv')

## To get other 4 fold Data

In [ ]:
#fold 1: id_test, id_val, id_train
except_fold1 = pd.concat([not_readmit_ID_use, readmit_ID, id_test, id_val])

In [ ]:
except_fold1 = except_fold1.drop_duplicates(keep=False)

In [ ]:
except_fold1_f = (pd.Index(except_fold1).intersection(pd.Index(readmit_ID))).values
except_fold1_t = (pd.Index(except_fold1).intersection(pd.Index(not_readmit_ID_use))).values

In [ ]:
len(except_fold1)

4740

In [ ]:
len(except_fold1_f)

2370

In [ ]:
folds = except_fold1.reset_index(drop=True)

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits = 4, shuffle = True, random_state = 1)
kf.get_n_splits(except_fold1_f)

4

In [ ]:
re={}
not_re={}
for i, (train_index, test_index) in  enumerate(kf.split(except_fold1_f)):
    #print("TRAIN:", train_index) 
    #print("TEST:", test_index)
    not_re[i+1] = except_fold1_f[test_index]

In [ ]:
len(not_re[2])

593

In [ ]:
kf.get_n_splits(except_fold1_t)
for i, (train_index, test_index) in  enumerate(kf.split(except_fold1_t)):
    #print("TRAIN:", train_index) 
    #print("TEST:", test_index)
    re[i+1] = except_fold1_t[test_index]

In [ ]:
sum(df_adm1[df_adm1.HADM_ID.isin(not_re[2])].OUTPUT_LABEL)

593

In [ ]:
np.intersect1d(re[1],re[4])

array([], dtype=int64)

In [ ]:
val_id_folds={}
test_id_folds={}

# Add fold1 for val
val_id_fold1_t = pd.Series(pd.Index(id_val).intersection(pd.Index(readmit_ID)).values)
val_id_fold1_f = pd.Series(pd.Index(id_val).intersection(pd.Index(not_readmit_ID_use)).values)

val_id_folds[0] = pd.concat([val_id_fold1_t, val_id_fold1_f]).values

# Add fold1 for test
test_id_fold1_t = pd.Series(pd.Index(id_test).intersection(pd.Index(readmit_ID)).values)
test_id_fold1_f = pd.Series(pd.Index(id_test).intersection(pd.Index(not_readmit_ID_use)).values)

test_id_folds[0] = pd.concat([test_id_fold1_t, test_id_fold1_f]).values

for i in range(1,5):
    ind_r = pd.Series(re[i]).sample(frac=0.5,random_state=1)
    d = pd.concat([ind_r, pd.Series(re[i])])
    ind_r1 = d.drop_duplicates(keep=False)
    ind_nr = pd.Series(not_re[i]).sample(frac=0.5,random_state=1)
    d = pd.concat([ind_nr, pd.Series(not_re[i])])
    ind_nr1 = d.drop_duplicates(keep=False)
    
    val_id_folds[i] = pd.concat([ind_r, ind_nr]).values
    test_id_folds[i] = pd.concat([ind_r1, ind_nr1]).values

In [ ]:
np.intersect1d(test_id_folds[0],val_id_folds[0])

array([], dtype=int64)

In [ ]:
sum(df_adm1[df_adm1.HADM_ID.isin(test_id_folds[0])].OUTPUT_LABEL)

297

In [ ]:
train_id_folds={}
for i in range(5):
    all_index = pd.concat([not_readmit_ID_use, readmit_ID, pd.Series(test_id_folds[i]), pd.Series(val_id_folds[i])])
    train_id_folds[i] = all_index.drop_duplicates(keep=False).values                             

In [ ]:
np.intersect1d(test_id_folds[3],train_id_folds[3])

array([], dtype=int64)

## folds discharge

# adjust the i in the following block to get different fold. Here, i = 0 means fold2, i=1 means fold 3, ... because we get fold 1 without using the Kfold library

In [ ]:
want=pd.read_csv('./readmission_task.csv')
df = pd.concat([not_readmit_ID_use, not_readmit_ID])
df = df.drop_duplicates(keep=False)

i=0
discharge_train = want[want.ID.isin(train_id_folds[i])]
discharge_val = want[want.ID.isin(val_id_folds[i])]
discharge_test = want[want.ID.isin(test_id_folds[i])]

discharge_train.Label.value_counts()

1.0    13134
0.0    11262
Name: Label, dtype: int64

In [ ]:
discharge_test.Label.value_counts()

1.0    1697
0.0    1366
Name: Label, dtype: int64

In [ ]:
dis_train = want[want.ID.isin(df.sample(n=437, random_state=1))]
len(dis_train)

2015

In [ ]:
(pd.Index(dis_train.ID).intersection(pd.Index(discharge_train.ID))).values


array([], dtype=float64)

In [ ]:
discharge_train_snippets = pd.concat([dis_train, discharge_train])
discharge_train_snippets = discharge_train_snippets.sample(frac=1, random_state=1).reset_index(drop=True)

In [ ]:
file = './good_datasets/fold'+str(i+2)+'/discharge/train.csv'
discharge_train_snippets.to_csv(file)

In [ ]:
file = './good_datasets/fold'+str(i+2)+'/discharge/val.csv'
discharge_val.to_csv(file)
file = './good_datasets/fold'+str(i+2)+'/discharge/test.csv'
discharge_test.to_csv(file)

## folds Early Notes

In [ ]:
want_early = pd.read_csv('./less_3_days_notes.csv')

In [ ]:
#want_2days = pd.read_csv('./less_2_days_notes.csv')

# want_2days = pd.read_csv('./data/2days/test.csv')
actionable_ID_3days = df_adm1[df_adm1['STAY_DAYS'] >= 3].HADM_ID
actionable_ID_2days = df_adm1[df_adm1['STAY_DAYS'] >= 2].HADM_ID

In [ ]:
i=3
early_train = want_early[want_early.ID.isin(train_id_folds[i])]
early_val = want_early[want_early.ID.isin(val_id_folds[i])]
test_actionable_id_3days = pd.Series(test_id_folds[i])[pd.Series(test_id_folds[i]).isin(actionable_ID_3days)]
early_test = want_early[want_early.ID.isin(test_actionable_id_3days)]

early_train.Label.value_counts()

1.0    24485
0.0    20266
Name: Label, dtype: int64

In [ ]:
early_test.Label.value_counts()

1.0    2523
0.0    2295
Name: Label, dtype: int64

In [ ]:
ear_train = want_early[want_early.ID.isin(df.sample(n=400, random_state=1))]
len(ear_train)

3161

In [ ]:
early_train_snippets = pd.concat([ear_train, early_train])
early_train_snippets = early_train_snippets.sample(frac=1, random_state=1).reset_index(drop=True)

In [ ]:
early_train_snippets.Label.value_counts()

1.0    24485
0.0    23427
Name: Label, dtype: int64

In [ ]:
file = './good_datasets/fold'+str(i+2)+'/3days/train.csv'
early_train_snippets.to_csv(file)

In [ ]:
file = './good_datasets/fold'+str(i+2)+'/3days/val.csv'
early_val.to_csv(file)
file = './good_datasets/fold'+str(i+2)+'/3days/test.csv'
early_test.to_csv(file)

In [ ]:
(pd.Index(early_val.TEXT).intersection(pd.Index(early_train_snippets.TEXT))).values

array([], dtype=object)

In [ ]:
test_actionable_id_2days = pd.Series(test_id_folds[i])[pd.Series(test_id_folds[i]).isin(actionable_ID_2days)]

In [ ]:
len(test_actionable_id_2days)

559

In [ ]:
early_test_2days = want_2days[want_2days.ID.isin(test_actionable_id_2days)]
early_test_2days.Label.value_counts()

1.0    1810
0.0    1809
Name: Label, dtype: int64

In [ ]:
early_test_2days.to_csv('./good_datasets/fold'+str(i+2)+'/2days/test.csv')